# Import Dependencies

In [12]:
import re
import pickle
import string
import numpy as np
import pandas as pd

# import spacy
# from spacy.lang.en.stop_words import STOP_WORDS
# from spacy.attrs import IS_ALPHA
# from spacy.lang.en import English
# from spacy import displacy, lemmatizer

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

import matplotlib.pyplot as plt

# from wordcloud import WordCloud
# from textwrap import wrap

import pyLDAvis
import pyLDAvis.sklearn

# Custom functions from .py files
from web_scrape import get_transcript
from preprocess import clean, preprocess

# Import/Preprocess Data

In [13]:
# Web scrape most recent speeches using our imported 'get_transcript' function
goodyear = get_transcript('https://www.rev.com/blog/transcripts/donald-trump-rally-speech-transcript-goodyear-az-october-28')
bullhead = get_transcript('https://www.rev.com/blog/transcripts/donald-trump-rally-speech-transcript-bullhead-city-az-october-28')
omaha = get_transcript('https://www.rev.com/blog/transcripts/donald-trump-rally-speech-transcript-omaha-ne-october-27')
wsalem = get_transcript('https://www.rev.com/blog/transcripts/donald-trump-rally-speech-transcript-west-salem-wisconsin-october-27')
lansing = get_transcript('https://www.rev.com/blog/transcripts/donald-trump-rally-speech-transcript-lansing-michigan-october-27')
martinsburg = get_transcript('https://www.rev.com/blog/transcripts/donald-trump-rally-speech-transcript-martinsburg-pa-october-26')
lititz = get_transcript('https://www.rev.com/blog/transcripts/donald-trump-rally-speech-transcript-lititz-pa-october-26')
allentown = get_transcript('https://www.rev.com/blog/transcripts/donald-trump-rally-speech-transcript-allentown-pa-october-26')

In [14]:
# Create corpus
corpus = [goodyear, bullhead, omaha, wsalem, lansing, martinsburg, lititz, allentown]

In [15]:
# Clean corpus using our imported 'clean' function
clean_corpus = clean(corpus)

In [16]:
# Preprocess the corpus using our imported 'preprocess' function
processed_corpus = preprocess(clean_corpus)

# Doc-Term Matrix

In [17]:
# Countvectorizer
cv = CountVectorizer(stop_words='english')

X_cv = cv.fit_transform(processed_corpus)

df_cv = pd.DataFrame(X_cv.toarray(), columns=cv.get_feature_names())

In [18]:
# TF_IDF
tfidf_vectorizer = TfidfVectorizer(**cv.get_params())

X_tfidf = tfidf_vectorizer.fit_transform(processed_corpus)

df_tfidf = pd.DataFrame(X_tfidf.toarray(), columns=tfidf_vectorizer.get_feature_names())

# LDA

In [19]:
# for TF DTM
lda_cv = LatentDirichletAllocation(n_components=5, random_state=0)
lda_cv.fit(df_cv)

lda_tfidf = LatentDirichletAllocation(n_components=5, random_state=0)
lda_tfidf.fit(df_tfidf)

LatentDirichletAllocation(n_components=5, random_state=0)

In [20]:
# LDA Visualization for CountVectorizer
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda_cv, X_cv, cv)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.101070  0.121648       1        1  22.228681
3      0.137826 -0.234063       2        1  21.455729
4     -0.079850  0.163331       3        1  20.481303
1      0.110658  0.050674       4        1  19.097359
0     -0.269704 -0.101591       5        1  16.736929, topic_info=         Term        Freq       Total Category  logprob  loglift
1298     know  451.000000  451.000000  Default  30.0000  30.0000
2030    right  394.000000  394.000000  Default  29.0000  29.0000
1715   people  335.000000  335.000000  Default  28.0000  28.0000
2693     year  279.000000  279.000000  Default  27.0000  27.0000
2646      win  242.000000  242.000000  Default  26.0000  26.0000
...       ...         ...         ...      ...      ...      ...
501   country   40.316152  223.386802   Topic5  -4.8135   0.0754
986       god   23.557850   39.489833   Topic5  -5.3508   1.2710
1354      let   26.889176  117.700203   Topic5  -5.2185   0.3111
2409    texas   22.762502   34.965785   Topic5  -5.3851   1.3583
2280   states   22.796389   38.112804   Topic5  -5.3836   1.2736

[279 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
50        1  0.981796      ago
50        5  0.011973      ago
73        5  0.944046  amazing
76        1  0.790500  america
76        3  0.151372  america
...     ...       ...      ...
2673      3  0.014995    world
2673      4  0.959662    world
2686      3  0.972579    wrong
2692      2  0.970797     yeah
2693      1  0.995794     year

[383 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 4, 5, 2, 1])

In [14]:
# LDA Visualization for TF_IDF
pyLDAvis.sklearn.prepare(lda_tfidf, X_tfidf, tfidf_vectorizer)

PreparedData(topic_coordinates=              x    y  topics  cluster      Freq
topic                                          
1      0.191172  0.0       1        1  51.15475
0     -0.191172  0.0       2        1  48.84525, topic_info=              Term        Freq       Total Category  logprob  loglift
2030         right  201.000000  201.000000  Default  30.0000  30.0000
1008         great  152.000000  152.000000  Default  29.0000  29.0000
2417         think  142.000000  142.000000  Default  28.0000  28.0000
991           good  121.000000  121.000000  Default  27.0000  27.0000
1715        people  116.000000  116.000000  Default  26.0000  26.0000
...            ...         ...         ...      ...      ...      ...
1168    incredible   27.183447   30.099645   Topic2  -5.5618   0.6146
564            day   34.712683   45.754747   Topic2  -5.3173   0.4403
223          biden   32.884145   62.413682   Topic2  -5.3714   0.0757
1712  pennsylvania   30.284019   48.307259   Topic2  -5.4538   0.2496
1298          know   31.914682  214.128915   Topic2  -5.4014  -1.1870

[115 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
50        1  0.972004      ago
50        2  0.030375      ago
76        1  0.690864  america
76        2  0.302253  america
162       1  0.967252      bad
...     ...       ...      ...
2670      2  0.024311     work
2673      1  0.989938    world
2673      2  0.024145    world
2693      1  0.982053     year
2693      2  0.020042     year

[147 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1])